In [1]:
import scanpy as sc
import pandas as pd

## put count layer into adata.X

In [2]:
rna = sc.read_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13.h5ad')

In [4]:
rna.X = rna.layers['UMIs']

In [5]:
rna.X.data

array([1., 1., 1., ..., 1., 1., 2.], dtype=float32)

In [6]:
rna.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_count.h5ad')

In [2]:
rna_dlpfc = sc.read_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_DLPFC_RNAseq_final-nuclei.2024-02-13.h5ad')

In [3]:
rna_dlpfc.X = rna_dlpfc.layers['UMIs']

In [4]:
rna_dlpfc.X.data

array([1., 2., 1., ..., 1., 1., 1.], dtype=float32)

In [5]:
rna_dlpfc.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_DLPFC_RNAseq_final-nuclei.2024-02-13_count.h5ad')

## concatenating rosmap data together

In [ ]:
def read_with_dask(path):
    with h5py.File(path, "r") as f:
        adata = ad.AnnData(
            obs=ad.io.read_elem(f["obs"]),
            var=ad.io.read_elem(f["var"]),
        )
        adata.X = ad.experimental.read_elem_as_dask(
            f["X"], chunks=(SPARSE_CHUNK_SIZE, adata.shape[1])
        )
    adata.var = adata.var.set_index('_index')
    return adata

In [ ]:
microglia = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/microglia.h5ad')
inhibitory = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/inhibitory.h5ad')
cux2_1 = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/cux2+.h5ad')
cux2_2 = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/cux2-.h5ad')
astrocytes = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/astrocytes.h5ad')
oligo = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/oligodendroglia.h5ad')
vascular = read_with_dask('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/vascular.niche.h5ad')


In [ ]:
all = ad.concat([microglia, inhibitory, cux2_1, cux2_2, astrocytes, oligo, vascular])
clinical = pd.read_csv('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/metadata/ROSMAP_clinical.csv')
all.obs = pd.merge(all.obs, clinical, on = 'individualID', how = 'left')
all.obs['class'] = all.obs['class'].astype(str)
all.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/all.h5ad')

## dask process to merge seaad and rosmap, and finding hvg

In [1]:
import dask.distributed as dd
import scanpy as sc
import anndata as ad
import h5py

sc.logging.print_header()

/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


scanpy==1.10.4 anndata==0.11.3 umap==0.5.7 numpy==2.1.3 scipy==1.15.1 pandas==2.2.3 scikit-learn==1.6.1 statsmodels==0.14.4 pynndescent==0.5.13


In [2]:
cluster = dd.LocalCluster(n_workers=2)
client = dd.Client(cluster)
SPARSE_CHUNK_SIZE = 10_000
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 20,Total memory: 500.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8241,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 500.00 GiB
Comm: tcp://127.0.0.1:8027,Total threads: 10
Dashboard: http://127.0.0.1:8071/status,Memory: 250.00 GiB
Nanny: tcp://127.0.0.1:8049,


In [14]:
%%time
# read RNA data from MTG SEAAD
cell_atlas_path = '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_count.h5ad'

with h5py.File(cell_atlas_path, "r") as f:
    mtg = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=ad.io.read_elem(f["var"]),
    )
    mtg.X = ad.experimental.read_elem_as_dask(
        f["X"], chunks=(SPARSE_CHUNK_SIZE, mtg.shape[1])
    )

mtg.layers["counts"] = mtg.X.copy()  # Making sure we keep access to the raw counts

# read RNA data from DLPFC SEAAD
cell_atlas_path = '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_DLPFC_RNAseq_final-nuclei.2024-02-13_count.h5ad'
with h5py.File(cell_atlas_path, "r") as f:
    dlpfc = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=ad.io.read_elem(f["var"]),
    )
    dlpfc.X = ad.experimental.read_elem_as_dask(
        f["X"], chunks=(SPARSE_CHUNK_SIZE, dlpfc.shape[1])
    )

dlpfc.layers["counts"] = dlpfc.X.copy()  # Making sure we keep access to the raw counts

# read RNA data from ROSMAP
cell_atlas_path = '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/all.h5ad'
with h5py.File(cell_atlas_path, "r") as f:
    rosmap = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=ad.io.read_elem(f["var"]),
    )
    rosmap.X = ad.experimental.read_elem_as_dask(
        f["X"], chunks=(SPARSE_CHUNK_SIZE, rosmap.shape[1])
    )

rosmap.layers["counts"] = rosmap.X.copy()  # Making sure we keep access to the raw counts



/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


CPU times: user 6.59 s, sys: 4.11 s, total: 10.7 s
Wall time: 24.8 s


In [ ]:
mtg.obs = mtg.obs[[]]  
mtg.var = mtg.var[[]]  
mtg.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_count_empty.h5ad')

In [15]:
dlpfc.obs = dlpfc.obs[[]]  
dlpfc.var = dlpfc.var[[]] 
dlpfc.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_DLPFC_RNAseq_final-nuclei.2024-02-13_count_empty.h5ad')

In [17]:
rosmap.obs = rosmap.obs[[]]  
rosmap.var = rosmap.var[[]] 
rosmap.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/all_empty.h5ad')

In [3]:
%%time
data_dict = {
    'SEAAD_MTG': '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_MTG_RNAseq_final-nuclei.2024-02-13_count_empty.h5ad',
    'SEAAD_DLPFC': '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/SEAAD/SEAAD_DLPFC_RNAseq_final-nuclei.2024-02-13_count_empty.h5ad',
    'rosmap': '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/rosmap/dlpfc2/raw_h5ad/all_empty.h5ad'
}

ad.experimental.concat_on_disk(
    data_dict,
    '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/integrated_data/seaad_rosmap_rna.h5ad',
    join="inner", 
    merge = 'unique',
    label='dataset',
)

CPU times: user 9min 53s, sys: 11min 32s, total: 21min 25s
Wall time: 25min 12s


In [3]:
cell_atlas_path = '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/integrated_data/seaad_rosmap_rna.h5ad'

with h5py.File(cell_atlas_path, "r") as f:
    merged = ad.AnnData(
        obs=ad.io.read_elem(f["obs"]),
        var=ad.io.read_elem(f["var"]),
    )
    merged.X = ad.experimental.read_elem_as_dask(
        f["X"], chunks=(SPARSE_CHUNK_SIZE, merged.shape[1])
    )

merged.layers["counts"] = merged.X.copy()  # Making sure we keep access to the raw counts

In [4]:
%%time
sc.pp.normalize_total(merged)
sc.pp.log1p(merged)
sc.pp.highly_variable_genes(merged, n_top_genes=2000, subset = True)

CPU times: user 39.2 s, sys: 2.6 s, total: 41.8 s
Wall time: 6min 46s


In [7]:
merged.var

,highly_variable,means,dispersions,dispersions_norm
HES4,True,0.167952,1.215583,1.335504
VWA1,True,0.168614,1.414559,1.813780
HES5,True,0.030614,1.359211,1.680742
PRDM16-DT,True,0.097127,1.281577,1.494134
PRDM16,True,0.297813,1.812314,2.830228
...,...,...,...,...
MT-ND3,True,2.200129,3.349776,1.944375
MT-ND4L,True,0.620945,1.370025,1.316078
MT-ND4,True,1.969133,2.876593,1.593346
MT-ND5,True,0.967328,1.685301,1.532243


In [6]:
merged.var.to_csv('hvg.csv')

In [ ]:
%%time
merged.write_h5ad('/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/integrated_data/seaad_rosmap_rna_hvg.h5ad')

## calculating PCA and visualize a subset

In [7]:
merged

AnnData object with n_obs × n_vars = 4423484 × 2000
    obs: 'dataset'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [9]:
%%time
sc.pp.pca(merged, n_comps=300, mask_var=None)

IndexError: Index dimension must be 1 or 2